<a href="https://colab.research.google.com/github/HongYue1/LightNovel-Crawler-Colab/blob/main/lightnovel_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 <font color="#3399ff" size=10><b> How to use:</b></font>
### 1. Click on  <font color="yellow" size=5><b> connect </b></font>on the top right and wait untill it finishs connecting.
### 2. Before running a block, make sure to <font color="red" size=5> <b> fill the required data first. </b></font>
### 3. Run each block, wait for the block to finish before running the next block, the block finishes when you see a green check mark<font color ="#1bc757" size=5><b> ✔</b></font> next to the run button.
###4. After uploading finish you will be given the links as an output.
### 5. Note: to download several novels you don't need to redo all these steps, just download a novel then upload it, then go back to the download section and download the next novel then upload it....etc , <font color="red" size=5> <b> you can only upload one novel a time.</br></font>
### <font color="#00c3e3" size=6><b> Look at the buttom of this notebook for A list of tested and working sources.</b></font>
# <font color="FF1493" size=5> <b> Enjoy <3 </b> </font>
# <a href="https://github.com/HongYue1/LightNovel-Crawler-Colab">Github</a> <br >


## ⏬ Install LightNovel-crawler

In [ ]:
# @markdown <font size =5 color="yellow"> ⬅️ Click Here to install</font>
from google.colab import output
!pip install -U lightnovel-crawler
!pip install gofilepy-api

output.clear()
print("Done!")

## ⏬ Download Novel

In [ ]:
import asyncio
import os
import json
import requests
import threading
import urllib.parse
import subprocess
import shutil
import time
import uuid
from gofilepy import GofileClient

#@markdown <font size=5 color="yellow">⬅️ Click Here to Download and Upload (EPUB only)</font>

#@markdown <h4> Enter the Link:
Link = "" #@param {type:"string"}

#@markdown <h4> Enter the range of the chapters separated by a space or enter <font color="#3399ff" size=5><b>All</b></font> to download all chapters.
#@markdown <h5> Example to download chapters between 1 and 100: 1 100
Range = "All" #@param {type:"string"}


# Format = "epub" #@param ["epub", "pdf", "mobi", "docx", "txt"]
Format = "epub"
#@markdown <h4> Choose Upload Services:
Upload_to_Gofile = True #@param {type:"boolean"}
Upload_to_Buzzheavier = True #@param {type:"boolean"}
Upload_to_FilesVC = False #@param {type:"boolean"}

#@markdown <h4> Automatically clear downloads after upload?
Auto_Clean = True #@param {type:"boolean"}

def download_novel(link, format="epub", range="All"):
    """Download a novel from the provided link."""
    print(f"📚 Downloading novel from {link}...")

    # Create a unique output directory for this download
    download_id = str(uuid.uuid4())[:8]
    output_dir = f"/content/novels/{download_id}"
    os.makedirs(output_dir, exist_ok=True)

    misc_command = " -f --suppress"

    try:
        if range == "All":
            # Capture the return code to check if download succeeded
            result = !lncrawl --single -o $output_dir -s $link --format $format --all $misc_command
        else:
            result = !lncrawl --single -o $output_dir -s $link --format $format --range $range $misc_command

        # Check if files were actually downloaded (directory should have content)
        format_dir = os.path.join(output_dir, format)
        if not os.path.exists(format_dir) or not os.listdir(format_dir):
            print("⚠️ Download completed but no files were found")
            return None

        print("✅ Download completed!")
        return output_dir
    except Exception as e:
        print(f"❌ Download failed: {e}")
        # Clean up the empty directory
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
        return None

def get_downloaded_file(output_dir):
    """Get the path of the downloaded file."""
    try:
        if not output_dir or not os.path.exists(output_dir):
            raise FileNotFoundError("Output directory doesn't exist")

        format_dir = os.path.join(output_dir, Format)
        print("[DEBUG] Format directory: " + format_dir)

        if not os.path.exists(format_dir):
            raise FileNotFoundError("Format directory not found")

        files = os.listdir(format_dir)
        if not files:
            raise FileNotFoundError(f"No files found in {format_dir}")

        # Assuming there's only one file, take the first one
        file_path = os.path.join(format_dir, files[0])
        return file_path, files[0]
    except Exception as e:
        print(f"❌ Error getting file path: {e}")
        return None, None

def upload_to_gofile(file_path):
    """Upload file to Gofile.io."""
    try:
        if not file_path or not os.path.exists(file_path):
            raise FileNotFoundError("File not found for upload")

        print("📤 Uploading to Gofile...")
        client = GofileClient()
        with open(file_path, "rb") as file_obj:
            file = client.upload(file=file_obj)
        print("✅ Gofile upload complete!")
        return file.page_link
    except Exception as e:
        print(f"❌ Gofile upload error: {e}")
        return "Upload failed"

def upload_to_filesvc(file_path):
    """Upload file to files.vc."""
    try:
        if not file_path or not os.path.exists(file_path):
            raise FileNotFoundError("File not found for upload")

        print("📤 Uploading to Files.vc...")
        with open(file_path, "rb") as file_obj:
            files = {"file": file_obj}
            headers = {"X-Account-ID": "9215854213114742"}
            response = requests.post(
                "https://api.files.vc/upload", files=files, headers=headers, timeout=60
            )
            response.raise_for_status()
            print("✅ Files.vc upload complete!")
            return response.json()["file_url"]
    except Exception as e:
        print(f"❌ Files.vc upload error: {e}")
        return "Upload failed"

def upload_to_buzzheavier(file_path, filename):
    """Upload file to buzzheavier.com."""
    try:
        if not file_path or not os.path.exists(file_path):
            raise FileNotFoundError("File not found for upload")

        print("📤 Uploading to Buzzheavier...")
        buzzheavier_url = "https://w.buzzheavier.com/" + urllib.parse.quote(
            filename, safe=""
        )
        result = subprocess.run(
            ["curl", "-#o-", "-T", file_path, buzzheavier_url],
            capture_output=True,
            text=True,
            check=True,
            timeout=60,
        )
        json_data = json.loads(result.stdout)
        file_id = json_data["data"]["id"]
        print("✅ Buzzheavier upload complete!")
        return "https://buzzheavier.com/" + file_id
    except Exception as e:
        print(f"❌ Buzzheavier upload error: {e}")
        return "Upload failed"

def clean_downloads(output_dir):
    """Clean up downloaded files."""
    try:
        if output_dir and os.path.exists(output_dir):
            shutil.rmtree(output_dir)
            print("🧹 Downloads cleaned up")
    except Exception as e:
        print(f"❌ Error cleaning up: {e}")

def process_novel():
    """Main function to download and upload a novel."""
    if not Link:
        print("❌ Please provide a link to download")
        return

    # Download the novel to a unique directory
    output_dir = download_novel(Link, Format, Range)

    if not output_dir:
        print("❌ Download failed or no files were found. Please check the link and try again.")
        return

    # Wait a moment to ensure file is saved
    time.sleep(1)

    # Get the downloaded file
    file_path, filename = get_downloaded_file(output_dir)
    if not file_path:
        print("❌ Could not find file to upload")
        # Clean up the failed download directory
        clean_downloads(output_dir)
        return

    print(f"📁 Uploading file: {filename}")

    results = {}
    threads = []

    # Set up Gofile upload
    if Upload_to_Gofile:
        def gofile_thread_func():
            results["gofile"] = upload_to_gofile(file_path)

        gofile_thread = threading.Thread(target=gofile_thread_func)
        gofile_thread.daemon = True
        threads.append(gofile_thread)
        gofile_thread.start()

    # Set up FilesVC upload
    if Upload_to_FilesVC:
        def filesvc_thread_func():
            results["filesvc"] = upload_to_filesvc(file_path)

        filesvc_thread = threading.Thread(target=filesvc_thread_func)
        filesvc_thread.daemon = True
        threads.append(filesvc_thread)
        filesvc_thread.start()

    # Set up Buzzheavier upload
    if Upload_to_Buzzheavier:
        def buzzheavier_thread_func():
            results["buzzheavier"] = upload_to_buzzheavier(file_path, filename)

        buzzheavier_thread = threading.Thread(target=buzzheavier_thread_func)
        buzzheavier_thread.daemon = True
        threads.append(buzzheavier_thread)
        buzzheavier_thread.start()

    # Wait for all threads to complete (with timeout)
    for thread in threads:
        thread.join(timeout=120)

    # Check if any uploads were successful
    upload_succeeded = any(
        results.get(service) and results.get(service) != "Upload failed"
        for service in ["gofile", "filesvc", "buzzheavier"]
        if service in results
    )

    # Display results
    print("\n📊 Upload Results:")
    if Upload_to_Gofile:
        print(f"Gofile: {results.get('gofile', 'Upload timed out or failed')}")
    if Upload_to_FilesVC:
        print(f"Files.vc: {results.get('filesvc', 'Upload timed out or failed')}")
    if Upload_to_Buzzheavier:
        print(f"Buzzheavier: {results.get('buzzheavier', 'Upload timed out or failed')}")

    # Clean up if requested or if all uploads failed
    if Auto_Clean or not upload_succeeded:
        clean_downloads(output_dir)

    if upload_succeeded:
        print("\n✨ Process completed! You can now download another novel if needed.")
    else:
        print("\n⚠️ All uploads failed. The download directory has been cleaned up.")

# Run the process
process_novel()

## List of Working Sources sorted by speed.
⭐ = Personal Recommedation  

<font color="red" size="4"> Note:</font> <font size="3">working doesn't mean that it has the novel you want or the novel isn't missing chapters in that source, so always take notice of the the number of chapters while downloading.

In [ ]:
#@markdown ## <font color="yellow" size=6> ⬅️ Fetch working sources

import pandas as pd
from urllib.parse import urlparse, parse_qs
from IPython.display import display, Markdown, HTML
import re


def fetch_google_sheet(sheet_url):
    """
    Fetch data from Google Sheets URL and return as a DataFrame.

    Args:
        sheet_url (str): URL to Google Sheets document

    Returns:
        pandas.DataFrame: Data from the sheet
    """
    try:
        # Parse URL components
        parsed_url = urlparse(sheet_url)

        # Extract document ID from URL path
        path_parts = parsed_url.path.split("/")
        if len(path_parts) < 4:
            raise ValueError("Invalid Google Sheets URL format")
        doc_id = path_parts[3]

        # Extract gid from URL query parameters or fragment
        params = parse_qs(parsed_url.query)
        fragment = parse_qs(parsed_url.fragment)

        # Look for gid in both query params and fragment
        gid = params.get("gid", ["0"])[0]
        if gid == "0" and "gid" in fragment:
            gid = fragment.get("gid", ["0"])[0]

        # Create CSV export URL
        export_url = f"https://docs.google.com/spreadsheets/d/{doc_id}/export?format=csv&gid={gid}"

        # Read data from Google Sheets
        return pd.read_csv(export_url)

    except Exception as e:
        print(f"Error fetching Google Sheet: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error


def format_cell_content(value):
    """Format cell content for better display in HTML table."""
    if pd.isna(value):
        return ""

    # Convert to string
    value_str = str(value)

    # Format URLs as clickable links with target="_blank" to open in new tab
    url_pattern = r"https?://\S+"
    if re.match(url_pattern, value_str):
        # For Colab, using explicit HTML anchor tags instead of markdown links
        return f'<a href="{value_str}" target="_blank">{value_str}</a>'

    # Truncate long text
    if len(value_str) > 50:
        return value_str[:47] + "..."

    return value_str


def display_as_enhanced_html_table(df, max_rows=None):
    """
    Display DataFrame as a styled HTML table with enhanced formatting.

    Args:
        df (pandas.DataFrame): Data to display
        max_rows (int, optional): Maximum number of rows to display
    """
    if df.empty:
        display(HTML("<p><em>No data available</em></p>"))
        return

    # Limit rows if specified
    if max_rows and len(df) > max_rows:
        display_df = df.head(max_rows)
        footer_note = f"<p><em>Showing {max_rows} of {len(df)} rows</em></p>"
    else:
        display_df = df
        footer_note = ""

    # Generate styled HTML table for better control and compatibility with Colab
    html = "<style>\n"
    html += "table {border-collapse: collapse; width: 80%; margin: 15px 0;}\n"
    html += "th {background-color: #89b4fa; font-size: 18px; color: black; text-align: center; padding: 12px; white-space: nowrap; font-weight: bold;}\n"
    html += "td {background-color: #313244; font-size: 16px; border: 1px solid #45475a; padding: 8px; text-align: center;}\n"
    html += "</style>\n"

    html += "<table border='1'>\n"

    # Add header row
    html += "<tr>\n"
    for col in display_df.columns:
        html += f"<th>{col}</th>\n"
    html += "</tr>\n"

    # Add data rows with custom formatting for each cell
    for _, row in display_df.iterrows():
        html += "<tr>\n"
        for value in row:
            formatted_value = format_cell_content(value)
            html += f"<td>{formatted_value}</td>\n"
        html += "</tr>\n"

    html += "</table>\n"

    # Add footer note if needed
    if footer_note:
        html += footer_note

    # Display as HTML
    display(HTML(html))


def detect_and_format_numeric_columns(df):
    """Detect numeric columns and format them appropriately."""
    for col in df.columns:
        # Check if column contains numeric data
        if pd.api.types.is_numeric_dtype(df[col]):
            # Format integers without decimal places
            if df[col].apply(lambda x: x == int(x) if not pd.isna(x) else True).all():
                df[col] = (
                    df[col].fillna("").astype(str).replace(r"\.0$", "", regex=True)
                )
            # Format floats with 2 decimal places
            else:
                df[col] = df[col].apply(lambda x: f"{x:.2f}" if not pd.isna(x) else "")
    return df


def main():
    # Google Sheet URL
    input_url = "https://docs.google.com/spreadsheets/d/1SGfpvLFq7DzssklY90cmTKqQEBNeOBcAj1zSHDv-gfA/edit?usp=sharing"

    # Fetch data
    df = fetch_google_sheet(input_url)

    # Pre-process data for better display
    df = detect_and_format_numeric_columns(df)

    # Display the enhanced table
    display_as_enhanced_html_table(df, max_rows=500)


if __name__ == "__main__":
    main()
